In [86]:
import pandas as pd
import numpy as np
import math as m

In [87]:
# Here we are calculating the gain ratio.

def calculate_gain_ratio(total,total_x1,total_x2,entropy,entropy_x1,entropy_x2):
    info_gain=entropy-((total_x1/total)*entropy_x1+(total_x2/total)*entropy_x2)
    sum_1=0
    sum_2=0;
    if(total_x1!=0):
        cal_1=total/total_x1
        sum_1=(total_x1/total)*(m.log(cal_1,2))
    if(total_x2!=0):
        cal_2=total/total_x2
        sum_2=(total_x2/total)*(m.log(cal_2,2))
    split_info=sum_1+sum_2
    gain_ratio=info_gain/split_info
    return gain_ratio

In [88]:
# Here we are calculating the entropy .

def calculate_entropy(x1_0,x1_1,x2_0,x2_1):
    total_1=x1_1+x2_1
    total_0=x2_0+x1_0
    total=total_1+total_0
    sum_1=0
    sum_0=0
    if(total_1!=0):
        num_1=total/total_1
        sum_1=((total_1/total))*(m.log(num_1,2))
    if(total_0!=0):
        num_0=total/total_0
        sum_0=((total_0/total))*(m.log(num_0,2))
    entropy=sum_1+sum_0
    
    sum_1=0
    sum_0=0
    total_x1=x1_0+x1_1
    if(x1_1!=0):
        num_x1_1=total_x1/x1_1
        sum_1=((x1_1/total_x1))*(m.log(num_x1_1,2))
    if(x1_0!=0):
        num_x1_0=total_x1/x1_0
        sum_0=((x1_0/total_x1))*(m.log(num_x1_0,2))
    entropy_x1=sum_1+sum_0
    
    sum_1=0
    sum_2=0
    total_x2=x2_0+x2_1
    if(x2_1!=0):
        num_x2_1=total_x2/x2_1
        sum_1=((x2_1/total_x2))*(m.log(num_x2_1,2))
    if(x2_0!=0):
        num_x2_0=total_x2/x2_0
        sum_0=((x2_0/total_x2))*(m.log(num_x2_0,2))
    entropy_x2=sum_1+sum_0
    
    gain_ratio=calculate_gain_ratio(total,total_x1,total_x2,entropy,entropy_x1,entropy_x2)
    
    return gain_ratio,entropy,total_1,total_0
   

In [89]:
# Here we are just storing the count of various zero and ones as output on splitting.

def gain_(f,x_train,y_train,wall,df):
    x1=df[df[f]<wall]
    x2=df[df[f]>=wall]
    l1=list(x1.iloc[:,4])
    l2=list(x2.iloc[:,4])
    count_x1_0=l1.count(0)
    count_x1_1=l1.count(1)
    count_x2_0=l2.count(0)
    count_x2_1=l2.count(1)
    return calculate_entropy(count_x1_0,count_x1_1,count_x2_0,count_x2_1)

In [90]:
# Calculating the gain ratio for the continuous data.

def gain(f,x_train,y_train,df):
    s1=x_train[f] 
    s1=set(s1)  # Storing the unique values in the set s1.  
    list1=list(s1) # Converting into list.
    list1.sort()   # Sorting the list.
    max_gain=-5
    for i in range(len(list1)-1):
        wall=(list1[i]+list1[i+1])/2 # Taking the arithmetic mean between the two numbers.
        ans=gain_(f,x_train,y_train,wall,df) # Calculting the entropy and gain ratio using the gain function of the arithmetic mean.
        ans=list(ans)
        if(max_gain<=ans[0]):
            max_gain=ans[0]
            sol=list(ans)
            sol.append(wall) # Storing the best arithmetic mean.
    return sol

In [91]:
# Here we are choosing the best feature to split and then we are splitting our decesion tree. 

def decesion_tree(x_train,y_train,features,df,level):
    
    max_gain1=-4
    
    if len(set(y_train))==1: # If the classes are pure. ( Base case of the recursive function (decesion_tree))
        for f in features:
            ans_=gain(f,x_train,y_train,df) # Checking each and every feature and choosing the best among them.
            ans_=list(ans_)
            if max_gain1<=ans_[0]:
                max_gain1=ans_[0]
                sol_=list(ans_)
                sol_.append(f)
    
        feat_=sol_[5]
        features.remove(sol_[5]) # Removing  the feature we have chosen to split on the basis of it.
        
        print("Level ",level)
        print("Count of zero ",sol_[3])
        print("Count of one ",sol_[2])
        print("Current Entropy ",sol_[1])
        print("Splitting on the feature ",feat_," with Gain Ratio ",sol_[0])
        print()
    
        return 
    
    if(len(features)==0): # Here we are returing when no feature is left to split.
        return

    
    # BASE CASES ENDS.
    
    for i in features:
        ans1=gain(i,x_train,y_train,df) # Checking each and every feature and choosing the best among them.
        ans1=list(ans1)
        if max_gain1<=ans1[0]:
            max_gain1=ans1[0]
            sol1=list(ans1)
            sol1.append(i)
    wall=sol1[4]
    feat=sol1[5]
    df1=df[df[feat]<wall]
    df2=df[df[feat]>=wall]
    
    x1_train=df1.iloc[:,0:3]
    y1_train=df1.iloc[:,4]
    x2_train=df2.iloc[:,0:3]
    y2_train=df2.iloc[:,4]

    features.remove(feat) # Removing  the feature we have chosen to split on the basis of it.
    
    decesion_tree(x1_train,y1_train,features,df1,level+1) # We are recursively calling on the splitted data.
    decesion_tree(x2_train,y2_train,features,df2,level+1) # We are recursively calling on the splitted data.
    
    print("Level ",level)
    print("Count of zero ",sol1[3])
    print("Count of one ",sol1[2])
    print("Current Entropy ",sol1[1])
    print("Splitting on the feature ",feat," with Gain Ratio ",sol1[0])
    print()
    
    return 
    

In [92]:
# Here we are converting the string data into int data type.

def f(s):
    if(s=="Iris-setosa"):
        return 0
    else:
        return 1
    

In [93]:
iris=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data")  #Extracting iris data

df=iris.copy() # Creating df as a panda type and storing the copy of iris data in it.

df.columns=["sl","sw","pl","pw","flower_type"] # Creating column header of every column.

df["flower_type"]=df.flower_type.apply(f) # Changing string data into number so that it can be made easy to use the data.

features=df.columns # Storing the feature_names in the feature.

features=list(features)

features.remove("flower_type") # Removing flower_type as a feature as it is not a feature_name.

In [94]:
x_train=df.iloc[:,0:df.shape[1]-1] # Extracting  input training  data.

y_train=df.iloc[:,df.shape[1]-1] # Extracting  output training data.

decesion_tree(x_train,y_train,features,df,0) # Printing the information regarding the decesion tree.

Level  1
Count of zero  49
Count of one  0
Current Entropy  0.0
Splitting on the feature  pl  with Gain Ratio  0.0

Level  1
Count of zero  0
Count of one  100
Current Entropy  0.0
Splitting on the feature  sw  with Gain Ratio  0.0

Level  0
Count of zero  49
Count of one  100
Current Entropy  0.913756430937882
Splitting on the feature  pw  with Gain Ratio  1.0

